In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [16]:
# !spacy download en_core_web_lg

In [17]:
# !pip install tweet-preprocessor

In [2]:
df=pd.read_csv('~/Downloads/archive (3)/links.csv')

In [3]:
df1=pd.read_csv('~/Downloads/archive (3)/movies.csv')

In [4]:
df2=pd.read_csv('~/Downloads/archive (3)/ratings.csv')

In [5]:
df3=pd.read_csv('~/Downloads/archive (3)/tags.csv')

In [6]:
df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
df1.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
df2.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [9]:
df3.head()

,userId,movieId,tag,timestamp
0,19,2324,bittersweet,1428651158
1,19,2324,holocaust,1428651112
2,19,2324,World War II,1428651118
3,23,7075,hilarious,1378675786
4,23,7075,Underrated,1378675786


In [10]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [11]:
spacy_tokenizer = nlp.tokenizer

In [217]:
df5= pd.merge(df3, df1, on='movieId', how='inner')

In [218]:
df5

,userId,movieId,tag,timestamp,title,genres
0,19,2324,bittersweet,1428651158,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
1,19,2324,holocaust,1428651112,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
2,19,2324,World War II,1428651118,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
3,4431,2324,emotional,1416754302,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
4,4431,2324,father-son relationship,1416754358,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
...,...,...,...,...,...,...
586989,247424,5467,Hell,1138054315,Never Again (2001),Comedy|Romance
586990,247424,5467,no!,1138054315,Never Again (2001),Comedy|Romance
586991,247509,95581,aviation,1434604017,"Flying Fleet, The (1929)",Adventure|Drama|Romance
586992,247509,95581,navy,1434604017,"Flying Fleet, The (1929)",Adventure|Drama|Romance


In [219]:
list(df5[df5['title']=='Never Again (2001)'].tag)[0]

'Hell'

In [44]:
# from tqdm import tqdm
# tqdm.pandas() 
# x=[spacy_tokenizer(str(x)) for x in df5.tag]

# vectors=[]
# for token in tqdm(x):
#     vectors.append(nlp(token).vector)
    


100%|██████████████████████████████████| 586994/586994 [22:30<00:00, 434.63it/s]


In [97]:
# import pickle
# with open ('vector.pickle', 'wb') as f:
#     pickle.dump(vectors, f, 5)

In [16]:
import pickle
with open('vector.pickle', 'rb') as f:
    data=pickle.load(f)

In [220]:
df4=pd.Series(data)

In [221]:
frames=[df5,df4]
df5=pd.concat(frames, axis=1)

In [222]:
df5.rename(columns={0: 'vec'}, inplace=True)

In [296]:
q= str(df5[df5['title']=='Never Again (2001)'].tag)

In [339]:
def prep(x):
    z=spacy_tokenizer(x)
    z=nlp(z).vector.reshape(300,)
    return z


In [377]:
def prepare(x):
    z=spacy_tokenizer(x)
    z=nlp(z).vector.reshape(300,)
    return z

In [436]:
q= str(df5[df5['title']=='Never Again (2001)'].tag)
def rec_movie():
    z=prep(q)
    listA=[]
    t=[prepare(str(x)) for x in df5.tag.loc[:]]
    for i in range(len(t)):
        listA.append(np.dot(t[i], z)/(np.linalg.norm(t[i])*np.linalg.norm(z)))
    ser= listA
    df5.timestamp=ser

In [437]:
rec_movie()

/var/folders/48/3rfqt3q17zj4zsvn_9rh50br0000gn/T/ipykernel_54515/3482208125.py:7: RuntimeWarning: invalid value encountered in float_scalars
  listA.append(np.dot(t[i], z)/(np.linalg.norm(t[i])*np.linalg.norm(z)))


In [447]:
df5.timestamp.sort_values(ascending=False)[0:7]

586757    0.728382
586759    0.728052
586761    0.728052
586760    0.728052
586758    0.712822
530597    0.690752
404344    0.660925
Name: timestamp, dtype: float32

In [448]:
df5.loc[563331], df5[df5['title']=='Feast of July (1995)']

(userId                                                   61431
 movieId                                                  44511
 tag          'Lemme Get This Straight. You Have No Idea Who...
 timestamp                                             0.655933
 title                                Unknown White Male (2005)
 genres                                             Documentary
 vec          [-0.037949346, 0.16567169, -0.24979131, -0.116...
 Name: 563331, dtype: object,
         userId  movieId                    tag  timestamp  \
 586986  247424      167  Last Words connection   0.561513   
 
                        title genres  \
 586986  Feast of July (1995)  Drama   
 
                                                       vec  
 586986  [0.35600498, 0.4164, -0.14501333, -0.038696665...  )

In [417]:
df5.drop_duplicates(subset='movieId', inplace=True)

In [230]:
df5.shape

(21654, 7)

In [449]:
df5

,userId,movieId,tag,timestamp,title,genres,vec
0,19,2324,bittersweet,0.064956,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,"[0.65223, 0.92595, 0.23045, 0.16232, -0.36456,..."
421,23,7075,hilarious,0.283356,"Court Jester, The (1956)",Adventure|Comedy|Musical,"[-0.65168, -0.12708, -0.096462, -0.13232, -0.4..."
467,54,357,Garath,NaN,Four Weddings and a Funeral (1994),Comedy|Romance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
605,57,260,Science Fiction,0.324257,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,"[-0.30313998, -0.039408498, 0.16473, -0.096814..."
9221,120,109374,cinematography,0.056059,"Grand Budapest Hotel, The (2014)",Comedy|Drama,"[-0.23285, -0.15892, 0.78638, -0.70825, 0.3790..."
...,...,...,...,...,...,...,...
586986,247424,167,Last Words connection,0.561513,Feast of July (1995),Drama,"[0.35600498, 0.4164, -0.14501333, -0.038696665..."
586987,247424,2998,Haunting Endings,0.204764,Dreaming of Joseph Lees (1999),Drama|Romance,"[0.0813985, -0.014964998, -0.70358497, -0.1624..."
586988,247424,3540,Last Words connection,0.561513,Passion of Mind (2000),Drama|Mystery|Romance,"[0.35600498, 0.4164, -0.14501333, -0.038696665..."
586989,247424,5467,Hell,0.514120,Never Again (2001),Comedy|Romance,"[-0.48299, -0.16004, -0.037345, -0.48164, 0.00..."
